<a href="https://colab.research.google.com/github/myrandaGoesToSpace/semantic-search-datasets/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers
!pip install bs4

     |████████████████████████████████| 78 kB 2.0 MB/s 
     |████████████████████████████████| 3.5 MB 5.4 MB/s 
     |████████████████████████████████| 6.8 MB 17.2 MB/s 
     |████████████████████████████████| 1.2 MB 30.0 MB/s 
     |████████████████████████████████| 67 kB 1.1 MB/s 
     |████████████████████████████████| 895 kB 18.8 MB/s 
     |████████████████████████████████| 596 kB 22.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=6395bcb34ad69007c358b0a26a13f0bb218c169762ac2cb6ea2703eabd6c96ec
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from sentence_transformers import SentenceTransformer, util
from bs4 import BeautifulSoup

In [ ]:
'''
# Example - semantic search

query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)


#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)
  
'''

In [5]:
import pandas as pd
import requests

def find_abstracts(soup):
  #df = pd.DataFrame(columns = ["identifier", "abstract"])
  id_list = []
  abs_list = []
  title_list = []

  for record in soup.find_all("csw:record"):
    id = record.find("dc:identifier")
    abs = record.find("dct:abstract")
    title = record.find("dc:title")

    # append id and abs to df
    #df = df.append([id.text, abs.text])
    id_list.append(id.text)
    title_list.append(title.text)

    if abs != None:
      abs_list.append(abs.text)
    else:
      abs_list.append("NA")

  return id_list, title_list, abs_list

# Get the abstracts from Geoportal
URL = "https://www.ncei.noaa.gov/metadata/geoportal/opensearch?f=csw&from=0&size=100&sort=title.sort"

page = requests.get(URL)
soup = BeautifulSoup(page.text, "lxml")

id_list, title_list, abs_list = find_abstracts(soup)
df = pd.DataFrame(list(zip(id_list,title_list, abs_list)), columns = ["identifier", "title", "abstract"])
df

,identifier,title,abstract
0,gov.noaa.nodc:0000662,(NCEI Accession 0000662),NA
1,gov.noaa.nodc:0208737,(NCEI Accession 0208737),NA
2,gov.noaa.nodc:0215155,(NCEI Accession 0215155),NA
3,gov.noaa.ngdc.mgg.geophysics:G01414,1-deg x 1-deg Terrestrial Mean Free-Air Anomalies,The 1x1 degree Terrestrial Mean Free-Air Gravi...
4,gov.noaa.nodc:0002166,11 Real-time XBT replacements assembled by Can...,NA
...,...,...,...
95,ACUMEN2012,ACUMEN 2012: Atlantic Canyons Undersea Mapping...,"Between February and August 2012, a team of NO..."
96,gov.noaa.nodc:0231662,ADCP data collected aboard NOAA Ship Bell M. S...,This dataset includes ADCP data collected aboa...
97,gov.noaa.nodc:0226205,ADCP data collected aboard NOAA Ship Gordon Gu...,This dataset includes ADCP data collected aboa...
98,gov.noaa.nodc:0241017,ADCP data collected aboard NOAA Ship Nancy Fos...,This dataset includes ADCP data collected aboa...


In [7]:

# Make the abstracts the docs
docs_df = df[df["abstract"] != "NA"]
docs = list(docs_df["abstract"])
titles = list(docs_df["title"])

# Query
query = input("Enter your query: ")

# predict on a search query for data

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores, titles))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)


#Output passages & scores
for doc, score, title in doc_score_pairs[:10]:
    print(score, title, doc)
    print('\n')

Enter your query: coral health in the Atlantic Ocean
0.5344188809394836 A survey of selected coral and fish assemblages near the Waianae Ocean Outfall, Oahu, Hawaii, 1990-1999 (NCEI Accession 0000794) During 1990-1999, coral growth and fish abundance were monitored at stations located at and in the vicinity of the Waianae Ocean Outfall. Comparisons of results with fish surveys showed no significant differences in the species composition or relative abundances of fish populations at Station W-2 (the sunken ship Mahi), which is located 1.2 km south of the diffuser. Fish abundance and species richness increased at Station W- 3, which is located at the diffuser, from 1990 to 1995, decreased in 1996, and increased again in 1997 through 1999. At Station WW, an inshore station located 0.8 km from shore, fish were abundant and speciose on the armor rock covering the pipeline. The fish species seen inshore are comparable to fish species seen in similar (boulder) natural biotopes around Hawaii. 